### Code Hist.

 - CODE  
    &ensp; : KIER Data_전처리 (결측 제거)  
  - DATE  
    &ensp; 2023-10-12 Created  
    &ensp; 2023-11-21 Code 최신화  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 기존 모듈화된 함수 사용  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 기존 KIER 변환 관련 코드를 모두 통합  
    &ensp; 2024-01-19 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) HOT (온수 사용량) 전용 Code 작성  
    &ensp; 2024-01-30 Code 개선 후 백업  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 순시사용량 보간 코드 작성 (미완)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 백업 후 코드 최적화    
    &ensp; 2024-02-05 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 순시사용량 보간 코드 작성 (Method 02 1차 완성)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ## A. 순시값을 보간하는 방법은 3가지로 제안 가능  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    a) 단순 Linear 보간 (앞뒤 평균치)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##       [Not Recommanded] 매우 긴 구간의 결측을 커버하기에 부적절  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##                         (양 끝이 0인 경우, 해당 구간을 제대로 보간 불가)      
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    b) 직전 주 동일시간 사용량  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##       나쁘지 않은 방법이지만, 직전 주의 데이터가 존재하지 않을 경우 보간 불가  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##       (데이터 상 첫 주 케이스 등)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    c-1) 해당 동일년도/동일월/동일요일/동일시간의 평균 사용량  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##         나쁘지 않은 방법이지만, 과정이 조금 복잡해짐  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    c-2) 해당 동일 일시의 348 세대의 평균 사용량  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##         c-1보다 기대되는 적용 데이터량이 많으며,   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##         한파/폭염/우천 등의 예외적인 주간의 사용량 변동을 피할 수 있어서 오차를 줄일 수 있다고 판단됨.

    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ## B. 순시값을 보간하기 전 단계  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    a) Resample시, 시간이 NaN인 구간은 그 간격에서 비어있는 행이다.  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##       해당 행의 순시값을 NaN으로 처리   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    b-1) 시점 n의 적산량이 0이고 시점 n + 1의 적산량이 정상 경우  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##         해당 행의 순시값을 NaN으로 처리  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##    b-2) 시점 n의 순시량이 음수인 경우  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; ##         해당 행의 순시값을 NaN으로 처리  
    &ensp; 2024-02-07 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 결측 처리 부분 정확도 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) Raw Data 상 결측치가 15% 이상일 경우, 해당 데이터를 사용하지 않는 부분 추가  
    
 - DESC  
    &ensp; : int_Domain 코드의 변경에 따라,   
    &emsp; KIER 에너지 사용량 데이터를 동/층/세대별로 분리    

    &emsp; 또한, Raw Data과 세대별 데이터 추출 단계에서,  
    &emsp; 아래와 같은 작업 진행  

    &emsp; [수행 작업]  
    &emsp; 1) Date Column에 대한 유효성 검사 및 이상 Data에 대한 소거  
    &emsp; 1-1) Datetime 유효성 확인  
    &emsp; 1-2) Datetime 이상치 제거  
    &emsp; 2) HOUSE_ID_HO의 해시값을 int_house_num으로 변경 (용량 절감 목적)  
 - DATA  
    1) Period :   
    2) Interval :   
 - Link  
    &ensp; : 

## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

#### Module import

In [1]:
#region Basic_Import
## Basic
import os
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import math
import random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
## Import_Local
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC
from Src_Dev_Common import DEV_KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [4]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [5]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-02-24 22:17:01.023753
2024 / 2 / 24
22 : 17


In [6]:
## Dict_Domain
dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 99:"GAS"} ## GAS는 사용하지 않음.
int_domain = 3
str_domain = str(dict_domain[int_domain])

dict_col_accu = {0 : "ACTUAL_ACCU_EFF" ## ELEC
                 , 1 : "ACCU_HEAT" ## HEAT
                 , 2 : "ACCU_FLOW" ## WATER
                 , 3 : "ACCU" ## HOT 열량
                 , 4 : "ACCU" ## HOT 유량
                 , 99 : "ACCU_FLOW" ## GAS
                 }
str_col_accu = str(str_domain + "_" + str(dict_col_accu[int_domain]))

dict_col_inst = {0 : "INST_EFF" ## ELEC_INST_EFF
                , 1 : "INST_HEAT" ## HEAT_INST_HEAT
                , 2 : "INST_FLOW" ## WATER_INST_FLOW
                , 3 : "INST" ## HOT_HEAT_INST
                , 4 : "INST" ## HOT_FLOW_INST
                , 99 : "INST_FLOW" ## GAS_INST_FLOW
                } 
str_col_inst = str(str_domain + "_" + str(dict_col_inst[int_domain]))

## Directory Root
str_dirData = "../data/data_Energy_KIER/"
str_dir_raw = '../data/data_Energy_KIER/KIER_0_Raw/'
str_dirName_bld = '../data/data_Energy_KIER/KIER_1_BLD/'
str_dirName_f = '../data/data_Energy_KIER/KIER_2_F_' + str_domain + '/'
str_dirName_h = '../data/data_Energy_KIER/KIER_3_H_' + str_domain + '/'

## File
str_fileRaw = str('KIER_RAW_' + str_domain + '_2023-11-12.csv')
str_fileRaw_hList = str('KIER_hList_' + str_domain + '.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'Data_Population_SJ_2014_2018_1st_Interpolated.csv', 'Data_Population_SJ_2014_2018_D_1st_Interpolated.csv', 'Data_Population_SJ_2014_2018_H_1st_Interpolated.csv', 'DATE_1M_2023-10-20.csv', 'ENV_SENSOR_HST_202309251452.csv', 'F1_MAIN_POWER_HST_202309251503.csv', 'F1_SUB_POWER_HST_202309251510.csv', 'HEATPUMP_HST_202309251520.csv', 'KIER_0_Raw', 'KIER_1_BLD', 'KIER_2_F_ELEC', 'KIER_2_F_GAS', 'KIER_2_F_HEAT', 'KIER_2_F_HOT', 'KIER_2_F_WATER', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', 'LIGHT_HST_202309251523.csv', 'MACHINE_ROOM_HEAT_HST_202309251534.csv', 'MACHINE_ROOM_POWER_APT_HST_20221205_202309251617.csv', 'MACHINE_ROOM_POWER_APT_HST_20221205_202309

In [7]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList
                            , index_col = 0)
print(df_kier_hList.columns)
df_kier_hList

Index(['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID_HO'], dtype='object')


,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID_HO
0,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=
63302,561,1,2,ODBEOTE0QzZCMjg5RUMzNTRBQjM1NjE0RDNBNDE0NzU=
126605,561,1,3,QTUwRUJDQTlBMkZENUQyNjg0RDRGREVFMTNGNDI3NUY=
189893,561,1,4,RkRCQzY4OTlCMTRDQjgzNUZBREE3Njc2MkM2MUM2QjI=
252954,561,2,1,MTQxNkQ1RDAzOTE5NTM5RUM3MzlDQkY3RTI2NERCQTM=
...,...,...,...,...
21692970,563,23,2,QkJCQjQwOThFNjgwNjUwQzJGRUJFNDlCNTREOEUxNDU=
21756267,563,23,3,QUJENzFFMEFDRjM4MjY3NTRDQzJCRjdDNUQ3ODA0N0Y=
21819571,563,23,4,RUVENzZDNTc4MUY4NDg5MUREQjZEOUM1M0Q2RjYzMDE=
21882854,563,24,1,NEVENDc1OTFFNDBBRjBDQzkzMTlEMDgzMkMzMkZFQTg=


## 01-02. Data Load (df_raw)

### 01-02-01. KIER (Energy Usage)

### KIER 사용량 데이터 Load

In [8]:
df_h = pd.DataFrame()
df_h['HOUSE_LIST'] = df_kier_hList['HOUSE_ID_DONG'].astype('string') + '-' + df_kier_hList['HOUSE_ID_HO_PRE'].astype('string') + '-' + df_kier_hList['HOUSE_ID_HO_INT'].astype('string')
df_h = df_h.drop_duplicates().reset_index()[['HOUSE_LIST']]
df_h

,HOUSE_LIST
0,561-1-1
1,561-1-2
2,561-1-3
3,561-1-4
4,561-2-1
...,...
343,563-23-2
344,563-23-3
345,563-23-4
346,563-24-1


In [9]:
## 순시사용량
str_file = 'KIER_' + str_domain + '_INST_10MIN_MEANINTERPOLATED.csv'
df_inst_tmp = pd.read_csv(str_dirName_h + str_file
                        , index_col = 0)
df_inst_tmp['METER_DATE'] = pd.to_datetime(df_inst_tmp['METER_DATE'])
df_inst_tmp = df_inst_tmp.reset_index()
df_inst_tmp = df_inst_tmp.drop(columns=['level_0', 'index'
                                        , 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'
                                        , 'MEAN_OF_INST', 'SUM_OF_INST'])

## 적산사용량
str_file = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
df_accu_tmp = pd.read_csv(str_dirName_h + str_file
                        , index_col = 0)
df_accu_tmp['METER_DATE'] = pd.to_datetime(df_accu_tmp['METER_DATE'])
df_accu_tmp = df_accu_tmp.reset_index()
df_accu_tmp = df_accu_tmp.drop(columns=['index'
                                        , 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'
                                        , 'MEAN_OF_ACCU', 'SUM_OF_ACCU'])

## 적산값 데이터의 첫 값이 NaN인 경우, 향후 연산에서 전체 데이터에 대한 NaN값으로 연산 우려
## 해당 값을 해당 호실 적산 사용량의 최소치로 치환
list_col = df_accu_tmp.columns[1:]
for str_col in list_col:
    if pd.isna(df_accu_tmp[str_col].iloc[0]):
        print(str_col)
        # print(df_accu_tmp[str_col].isna().sum())
        # print(df_accu_tmp[str_col].min())
        df_accu_tmp[str_col].iloc[0] = df_accu_tmp[str_col].min()

## 최종 보간 및 연산된 적산 데이터
df_accu = df_accu_tmp
# df_accu

HOT_HEAT_ACCU_562-5-3
HOT_HEAT_ACCU_562-12-3
HOT_HEAT_ACCU_562-16-2
HOT_HEAT_ACCU_562-20-6
HOT_HEAT_ACCU_563-18-4


#### 완전히 보간된 순시 사용량을 통해 적산 사용량을 복원

In [10]:
for str_h in df_h['HOUSE_LIST']:
    for i in range(0, len(df_accu) - 1):
        df_accu[str_col_accu + '_' + str_h].loc[i + 1] = df_accu_tmp[str_col_accu + '_' + str_h].loc[i] + df_inst_tmp[str_col_inst + '_' + str_h].loc[i]
df_accu

,METER_DATE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,HOT_HEAT_ACCU_561-2-2,HOT_HEAT_ACCU_561-2-3,HOT_HEAT_ACCU_561-2-4,HOT_HEAT_ACCU_561-3-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022-07-17 23:20:00,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,2.4573000000,4.5127000000,3.2532000000,5.4223000000,...,11.0474000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2486000000,6.6785000000,4.8390000000,5.7506000000
1,2022-07-17 23:30:00,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,2.4573000000,4.5127000000,3.2532000000,5.4223000000,...,11.0476000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2486000000,6.6785000000,4.8390000000,5.7506000000
2,2022-07-17 23:40:00,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0477000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
3,2022-07-17 23:50:00,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0481000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
4,2022-07-18 00:00:00,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0483000000,7.6167000000,4.7249000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64739,2023-10-10 13:10:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3671712354,8.7438420612,8.3420333832
64740,2023-10-10 13:20:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3675712354,8.7438420612,8.3420333832
64741,2023-10-10 13:30:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3675712354,8.7438420612,8.3420333832
64742,2023-10-10 13:40:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3675712354,8.7438420612,8.3420333832


In [11]:
list_col = df_accu.columns[1:]

print(df_accu.shape)
for str_col in list_col:
    if df_accu[str_col].isnull().sum() > 0:
        print(str_col, '/', df_accu[str_col].isnull().sum(), '/', len(df_accu))
        # df_accu = df_accu.drop(columns = [str_col])
print(df_accu.shape)

(64744, 349)
(64744, 349)


In [12]:
str_file = 'KIER_' + str_domain + '_ACCU_10MIN_InstBaseUpdated.csv' 
df_accu.to_csv(str_dirName_h + str_file)

#### 각 시간단위 데이터 추출
10분단위 적산치 데이터를 통해  
1) 1시간 단위  
2-1) 1일 단위  
2-2) 평일/주말단위  
3) 1개월 단위  
4) 적분값 (적산 총량)  
값을 각각 산출  

In [13]:
str_file = 'KIER_' + str_domain + '_ACCU_10MIN_InstBaseUpdated.csv' 
df_accu = pd.read_csv(str_dirName_h + str_file
                      , index_col = 0)
df_accu

,METER_DATE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,HOT_HEAT_ACCU_561-2-2,HOT_HEAT_ACCU_561-2-3,HOT_HEAT_ACCU_561-2-4,HOT_HEAT_ACCU_561-3-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022-07-17 23:20:00,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,2.4573000000,4.5127000000,3.2532000000,5.4223000000,...,11.0474000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2486000000,6.6785000000,4.8390000000,5.7506000000
1,2022-07-17 23:30:00,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,2.4573000000,4.5127000000,3.2532000000,5.4223000000,...,11.0476000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2486000000,6.6785000000,4.8390000000,5.7506000000
2,2022-07-17 23:40:00,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0477000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
3,2022-07-17 23:50:00,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0481000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
4,2022-07-18 00:00:00,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0483000000,7.6167000000,4.7249000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64739,2023-10-10 13:10:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3671712354,8.7438420612,8.3420333832
64740,2023-10-10 13:20:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3675712354,8.7438420612,8.3420333832
64741,2023-10-10 13:30:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3675712354,8.7438420612,8.3420333832
64742,2023-10-10 13:40:00,8.7621799450,18.3302406914,12.4697291697,14.3572575678,-3763.9702938410,4.1935991108,9.2141606142,6.8909501687,9.1459799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8943342425,5.3126517676,7.3170235907,11.3675712354,8.7438420612,8.3420333832


In [14]:
## 10분별 순시
df_inst_resampled_10MIN = pd.DataFrame()
df_inst_resampled_10MIN['METER_DATE'] = df_accu['METER_DATE']

for str_h in df_h['HOUSE_LIST']:
    df_inst_resampled_10MIN[str_col_inst + '_' + str_h] = 0
    for i in range(0, len(df_accu) - 1):
        df_inst_resampled_10MIN[str_col_inst + '_' + str_h].loc[i] = df_accu[str_col_accu + '_' + str_h].loc[i + 1] - df_accu[str_col_accu + '_' + str_h].loc[i]
str_file = 'KIER_' + str_domain + '_INST_10MIN_InstBaseUpdated.csv' 
df_inst_resampled_10MIN.to_csv(str_dirName_h + str_file)
df_inst_resampled_10MIN

,METER_DATE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,HOT_HEAT_INST_561-1-4,HOT_HEAT_INST_561-2-1,HOT_HEAT_INST_561-2-2,HOT_HEAT_INST_561-2-3,HOT_HEAT_INST_561-2-4,HOT_HEAT_INST_561-3-1,...,HOT_HEAT_INST_563-22-3,HOT_HEAT_INST_563-22-4,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2
0,2022-07-17 23:20:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0002000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,2022-07-17 23:30:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0005000000,0.0000000000,0.0000000000,0.0000000000,...,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000
2,2022-07-17 23:40:00,0.0000000000,0.0000000000,0.0000000000,0.0002000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0004000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,2022-07-17 23:50:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0002000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
4,2022-07-18 00:00:00,0.0000000000,0.0000000000,0.0000000000,0.0027000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64739,2023-10-10 13:10:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004000000,0.0000000000,0.0000000000
64740,2023-10-10 13:20:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
64741,2023-10-10 13:30:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
64742,2023-10-10 13:40:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000


#### 1H : 1시간 단위

In [15]:
## 시간별 적산
df_accu_resampled_1H = com_Prep.resample_by_last(df_accu, str_domain, 'METER_DATE', '1H')
df_accu_resampled_1H = df_accu_resampled_1H.reset_index().drop(columns = ['METER_DATE']).rename(columns = {str_domain + '_METER_DATE' : 'METER_DATE'})

str_file = 'KIER_' + str_domain + '_ACCU_1H_InstBaseUpdated.csv' 
df_accu_resampled_1H.to_csv(str_dirName_h + str_file)
df_accu_resampled_1H

,METER_DATE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,HOT_HEAT_ACCU_561-2-2,HOT_HEAT_ACCU_561-2-3,HOT_HEAT_ACCU_561-2-4,HOT_HEAT_ACCU_561-3-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022-07-17 23:00:00,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0481000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
1,2022-07-18 00:00:00,5.1931000000,11.7151000000,6.8699000000,7.6745000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0484000000,7.6167000000,4.7249000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
2,2022-07-18 01:00:00,5.1931000000,11.7151038235,6.8699000000,7.6745183348,8.0506000000,2.4579000000,4.5127000000,3.2532000000,5.4223183348,...,11.0484000000,7.6168000000,4.7250000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
3,2022-07-18 02:00:00,5.1932000000,11.7151058824,6.8699000000,7.6745183348,8.0506000000,2.4579000000,4.5127000000,3.2532000000,5.4223183348,...,11.0484058395,7.6168000000,4.7250017910,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
4,2022-07-18 03:00:00,5.1932000000,11.7151058824,6.8699000000,7.6746183348,8.0506000000,2.4579000000,4.5127000000,3.2545000000,5.4223183348,...,11.0484058395,7.6168000000,4.7250053625,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10786,2023-10-10 09:00:00,8.7621799450,18.3286406914,12.4677291697,14.3572575678,-3763.9705938410,4.1934991108,9.2141606142,6.8900501687,9.1426799005,...,19.1334937967,-3765.1276282983,7.9491783814,4.6187382508,-3770.8951342425,5.3125626419,7.3170235907,11.3647712354,8.7425420612,8.3420333832
10787,2023-10-10 10:00:00,8.7621799450,18.3286406914,12.4677291697,14.3572575678,-3763.9705938410,4.1934991108,9.2141606142,6.8900501687,9.1426799005,...,19.1334937967,-3765.1276282983,7.9491783814,4.6187382508,-3770.8948342425,5.3125626419,7.3170235907,11.3648712354,8.7438420612,8.3420333832
10788,2023-10-10 11:00:00,8.7621799450,18.3286406914,12.4677291697,14.3572575678,-3763.9705938410,4.1934991108,9.2141606142,6.8904501687,9.1426799005,...,19.1334937967,-3765.1276282983,7.9512783814,4.6187382508,-3770.8948342425,5.3126517676,7.3170235907,11.3648712354,8.7438420612,8.3420333832
10789,2023-10-10 12:00:00,8.7621799450,18.3302406914,12.4677291697,14.3572575678,-3763.9702938410,4.1934991108,9.2141606142,6.8909501687,9.1433799005,...,19.1334937967,-3765.1276282983,7.9513783814,4.6187382508,-3770.8944342425,5.3126517676,7.3170235907,11.3670712354,8.7438420612,8.3420333832


In [16]:
## 시간별 순시
df_inst_resampled_1H = pd.DataFrame()
df_inst_resampled_1H['METER_DATE'] = df_accu_resampled_1H['METER_DATE']

for str_h in df_h['HOUSE_LIST']:
    df_inst_resampled_1H[str_col_inst + '_' + str_h] = 0
    for i in range(0, len(df_accu_resampled_1H) - 1):
        df_inst_resampled_1H[str_col_inst + '_' + str_h].loc[i] = df_accu_resampled_1H[str_col_accu + '_' + str_h].loc[i + 1] - df_accu_resampled_1H[str_col_accu + '_' + str_h].loc[i]
str_file = 'KIER_' + str_domain + '_INST_1H_InstBaseUpdated.csv' 
df_inst_resampled_1H.to_csv(str_dirName_h + str_file)
df_inst_resampled_1H

,METER_DATE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,HOT_HEAT_INST_561-1-4,HOT_HEAT_INST_561-2-1,HOT_HEAT_INST_561-2-2,HOT_HEAT_INST_561-2-3,HOT_HEAT_INST_561-2-4,HOT_HEAT_INST_561-3-1,...,HOT_HEAT_INST_563-22-3,HOT_HEAT_INST_563-22-4,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2
0,2022-07-17 23:00:00,0.0000000000,0.0000000000,0.0000000000,0.0028000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0003000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,2022-07-18 00:00:00,0.0000000000,0.0000038235,0.0000000000,0.0000183348,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000183348,...,0.0000000000,0.0001000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,2022-07-18 01:00:00,0.0001000000,0.0000020588,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000058395,0.0000000000,0.0000017910,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,2022-07-18 02:00:00,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0013000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000035714,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
4,2022-07-18 03:00:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10786,2023-10-10 09:00:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0003000000,0.0000000000,0.0000000000,0.0001000000,0.0013000000,0.0000000000
10787,2023-10-10 10:00:00,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004000000,0.0000000000,...,0.0000000000,0.0000000000,0.0021000000,0.0000000000,0.0000000000,0.0000891257,0.0000000000,0.0000000000,0.0000000000,0.0000000000
10788,2023-10-10 11:00:00,0.0000000000,0.0016000000,0.0000000000,0.0000000000,0.0003000000,0.0000000000,0.0000000000,0.0005000000,0.0007000000,...,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0004000000,0.0000000000,0.0000000000,0.0022000000,0.0000000000,0.0000000000
10789,2023-10-10 12:00:00,0.0000000000,0.0000000000,0.0020000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0026000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0005000000,0.0000000000,0.0000000000


#### 1D : 1일 단위

In [17]:
## 일별 적산
df_accu_resampled_1D = com_Prep.resample_by_last(df_accu, str_domain, 'METER_DATE', '1D')
df_accu_resampled_1D = df_accu_resampled_1D.reset_index().drop(columns = ['METER_DATE']).rename(columns = {str_domain + '_METER_DATE' : 'METER_DATE'})

str_file = 'KIER_' + str_domain + '_ACCU_1DAY_InstBaseUpdated.csv' 
df_accu_resampled_1D.to_csv(str_dirName_h + str_file)
df_accu_resampled_1D

,METER_DATE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,HOT_HEAT_ACCU_561-2-2,HOT_HEAT_ACCU_561-2-3,HOT_HEAT_ACCU_561-2-4,HOT_HEAT_ACCU_561-3-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022-07-17,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0481000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
1,2022-07-18,5.1944742462,11.7255282279,6.8781678356,7.6767204707,8.0523650054,2.4598469067,4.5175410783,3.2577804280,5.4251122415,...,11.0541481411,7.6214000000,4.7280302874,2.9404000000,3.2083144364,2.6513136506,4.2563434681,6.6825163752,4.8434481237,5.7506000000
2,2022-07-19,5.1965004991,11.7361298224,6.8841864904,7.6839387193,8.0594931956,2.4610073324,4.5217529365,3.2609223430,5.4285645543,...,11.0589283398,7.6235117606,4.7324673377,2.9415518214,3.2152111255,2.6570168628,4.2583434681,6.6859338011,4.8480892979,5.7506382461
3,2022-07-20,5.2002826889,11.7462149242,6.8916816201,7.6898073805,8.0624931956,2.4624186081,4.5293837117,3.2675171590,5.4333956038,...,11.0626117065,7.6289065766,4.7375673377,2.9425518214,3.2192158528,2.6638116787,4.2617434681,6.6881338011,4.8529892979,5.7507386503
4,2022-07-21,5.2033826889,11.7530183793,6.8972259039,7.6943912138,8.0627954911,2.4647328899,4.5338837117,3.2754641822,5.4373956505,...,11.0679410392,7.6334570410,4.7408014548,2.9434288637,3.2256193079,2.6647444273,4.2641434681,6.6914338011,4.8579892979,5.7507386503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,2023-10-06,8.7312513952,18.2755018176,12.4225245300,14.2994294532,-3764.0078987662,4.1801144425,9.1648462003,6.8449011467,9.1254857220,...,19.0573896102,-3765.1538952776,7.9302539542,4.6028373101,-3770.9112764442,5.2982861711,7.2867789178,11.3199468082,8.7249725327,8.3161742593
447,2023-10-07,8.7391234841,18.2865309139,12.4348406523,14.3173152429,-3764.0078266773,4.1840865315,9.1846182892,6.8561937079,9.1285578109,...,19.0840917447,-3765.1461370059,7.9371260431,4.6071050342,-3770.9052097034,5.3028582600,7.2908510068,11.3327188971,8.7311732221,8.3224564672
448,2023-10-08,8.7457784469,18.3110792700,12.4453070075,14.3287331880,-3763.9991513568,4.1868044766,9.1913362344,6.8710486706,9.1337127737,...,19.1021096899,-3765.1426190608,7.9423942745,4.6120229793,-3770.9029917583,5.3065987794,7.3048539622,11.3492871285,8.7336911672,8.3296032308
449,2023-10-09,8.7598877416,18.3275990921,12.4640155178,14.3496530102,-3763.9793428464,4.1890242987,9.2074560565,6.8843579653,9.1371220684,...,19.1275189846,-3765.1315773037,7.9476035693,4.6168336931,-3770.8962832479,5.3109210427,7.3154614799,11.3610964233,8.7395004620,8.3383107485


In [18]:
## 일별 순시
df_inst_resampled_1D = pd.DataFrame()
df_inst_resampled_1D['METER_DATE'] = df_accu_resampled_1D['METER_DATE']

for str_h in df_h['HOUSE_LIST']:
    df_inst_resampled_1D[str_col_inst + '_' + str_h] = 0
    for i in range(0, len(df_accu_resampled_1H) - 1):
        df_inst_resampled_1D[str_col_inst + '_' + str_h].loc[i] = df_accu_resampled_1H[str_col_accu + '_' + str_h].loc[i + 1] - df_accu_resampled_1H[str_col_accu + '_' + str_h].loc[i]
str_file = 'KIER_' + str_domain + '_INST_1DAY_InstBaseUpdated.csv' 
df_inst_resampled_1D.to_csv(str_dirName_h + str_file)
df_inst_resampled_1D

,METER_DATE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,HOT_HEAT_INST_561-1-4,HOT_HEAT_INST_561-2-1,HOT_HEAT_INST_561-2-2,HOT_HEAT_INST_561-2-3,HOT_HEAT_INST_561-2-4,HOT_HEAT_INST_561-3-1,...,HOT_HEAT_INST_563-22-3,HOT_HEAT_INST_563-22-4,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2
0,2022-07-17,0.0000000000,0.0000000000,0.0000000000,0.0028000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0003000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,2022-07-18,0.0000000000,0.0000038235,0.0000000000,0.0000183348,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000183348,...,0.0000000000,0.0001000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,2022-07-19,0.0001000000,0.0000020588,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000058395,0.0000000000,0.0000017910,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,2022-07-20,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0013000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000035714,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
4,2022-07-21,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,2023-10-06,0.0001000000,0.0000000000,0.0007000000,0.0000167183,0.0000000000,0.0000167183,0.0000000000,0.0000000000,0.0000167183,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000418698,0.0000000000
447,2023-10-07,0.0007000000,0.0005000000,0.0041000000,0.0000267042,0.0000000000,0.0000152239,0.0000000000,0.0000000000,0.0000414971,...,0.0001000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000
448,2023-10-08,0.0000000000,0.0000000000,0.0000090062,0.0000137830,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000335404,0.0002000000,0.0000000000,0.0000000000,0.0000000000
449,2023-10-09,0.0000000000,0.0000000000,0.0001112094,0.0000362292,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0006000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0012000000,0.0003000000,0.0000000000,0.0000000000


#### 1W : 1주 단위

In [19]:
## 일별 적산
df_accu_resampled_1W = com_Prep.resample_by_last(df_accu, str_domain, 'METER_DATE', '1W')
df_accu_resampled_1W = df_accu_resampled_1W.reset_index().drop(columns = ['METER_DATE']).rename(columns = {str_domain + '_METER_DATE' : 'METER_DATE'})

str_file = 'KIER_' + str_domain + '_ACCU_1WEEK_InstBaseUpdated.csv' 
df_accu_resampled_1W.to_csv(str_dirName_h + str_file)
df_accu_resampled_1W

,METER_DATE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,HOT_HEAT_ACCU_561-2-2,HOT_HEAT_ACCU_561-2-3,HOT_HEAT_ACCU_561-2-4,HOT_HEAT_ACCU_561-3-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022-07-17,5.1931000000,11.7151000000,6.8699000000,7.6717000000,8.0506000000,2.4578000000,4.5127000000,3.2532000000,5.4223000000,...,11.0481000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
1,2022-07-24,5.2099686723,11.7711543644,6.9089981255,7.7131706896,8.0754782738,2.4692350724,4.5455999325,3.2875522146,5.4466609847,...,11.0855921421,7.6570004089,4.7582650727,2.9462994498,3.2421622061,2.6713328307,4.2762141452,6.6997022386,4.8661681973,5.7509203129
2,2022-07-31,5.2298331444,11.8282410036,6.9409628122,7.7690230095,8.1070838688,2.4785151008,4.5648002824,3.3161990277,5.4690611614,...,11.1213442581,7.6877555695,4.7820541302,2.9587477383,3.2754375025,2.6883188530,4.2891311484,6.7208180004,4.8929090693,5.7512480679
3,2022-08-07,5.2512695403,11.8833092301,6.9737018521,7.8133165330,8.1186634835,2.4872659389,4.5884053345,3.3374530381,5.4866566848,...,11.1571025709,7.7083673820,4.8011421908,2.9713892518,3.3007309145,2.7017215605,4.3089205615,6.7412834754,4.9067616861,5.7526778217
4,2022-08-14,5.2692204761,11.9485413729,7.0153021345,7.8576567096,8.1197902675,2.4943631880,4.6107512309,3.3583966071,5.5058397262,...,11.1855207849,7.7326501646,4.8204673750,2.9841131609,3.3157996604,2.7151340199,4.3296758006,6.7630920190,4.9239922733,5.7530336399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2023-09-17,8.6051309998,18.0563641613,12.2308022042,14.0089820222,-3764.1961832051,4.1120544881,8.9810525021,6.6153040901,8.9849808128,...,18.6886016345,-3765.3037736948,7.8302957427,4.5579818900,-3771.0138738938,5.1924733563,7.1595818033,11.1032158616,8.5992001322,8.2322402968
62,2023-09-24,8.6466519961,18.1390492577,12.3144677241,14.1043703097,-3764.1339887603,4.1370902399,9.0497639902,6.6996896094,9.0370645507,...,18.8142929103,-3765.2561454112,7.8681300230,4.5771933759,-3770.9866402630,5.2435651625,7.2083932737,11.1726214514,8.6503900246,8.2604483830
63,2023-10-01,8.6920783566,18.2042707168,12.3680601954,14.2148027557,-3764.0727250448,4.1612264215,9.1212982715,6.7910176835,9.0846439673,...,18.9229280176,-3765.1942100768,7.8971479291,4.5997081895,-3770.9389023753,5.2663699207,7.2495214934,11.2613716759,8.6837510224,8.2931905304
64,2023-10-08,8.7457784469,18.3110792700,12.4453070075,14.3287331880,-3763.9991513568,4.1868044766,9.1913362344,6.8710486706,9.1337127737,...,19.1021096899,-3765.1426190608,7.9423942745,4.6120229793,-3770.9029917583,5.3065987794,7.3048539622,11.3492871285,8.7336911672,8.3296032308


In [20]:
## 시간별 순시
df_inst_resampled_1W = pd.DataFrame()
df_inst_resampled_1W['METER_DATE'] = df_accu_resampled_1W['METER_DATE']

for str_h in df_h['HOUSE_LIST']:
    df_inst_resampled_1W[str_col_inst + '_' + str_h] = 0
    for i in range(0, len(df_accu_resampled_1W) - 1):
        df_inst_resampled_1W[str_col_inst + '_' + str_h].loc[i] = df_accu_resampled_1W[str_col_accu + '_' + str_h].loc[i + 1] - df_accu_resampled_1W[str_col_accu + '_' + str_h].loc[i]
str_file = 'KIER_' + str_domain + '_INST_1WEEK_InstBaseUpdated.csv' 
df_inst_resampled_1W.to_csv(str_dirName_h + str_file)
df_inst_resampled_1W

,METER_DATE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,HOT_HEAT_INST_561-1-4,HOT_HEAT_INST_561-2-1,HOT_HEAT_INST_561-2-2,HOT_HEAT_INST_561-2-3,HOT_HEAT_INST_561-2-4,HOT_HEAT_INST_561-3-1,...,HOT_HEAT_INST_563-22-3,HOT_HEAT_INST_563-22-4,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2
0,2022-07-17,0.0168686723,0.0560543644,0.0390981255,0.0414706896,0.0248782738,0.0114350724,0.0328999325,0.0343522146,0.0243609847,...,0.0374921421,0.0403004089,0.0334650727,0.0081994498,0.0378622061,0.0247328307,0.0275141452,0.0212022386,0.0271681973,0.0003203129
1,2022-07-24,0.0198644721,0.0570866392,0.0319646867,0.0558523199,0.0316055950,0.0092800284,0.0192003499,0.0286468131,0.0224001767,...,0.0357521160,0.0307551605,0.0237890575,0.0124482886,0.0332752965,0.0169860223,0.0129170031,0.0211157618,0.0267408720,0.0003277550
2,2022-07-31,0.0214363959,0.0550682265,0.0327390398,0.0442935235,0.0115796147,0.0087508382,0.0236050522,0.0212540104,0.0175955234,...,0.0357583129,0.0206118126,0.0190880606,0.0126415135,0.0252934120,0.0134027074,0.0197894131,0.0204654750,0.0138526168,0.0014297537
3,2022-08-07,0.0179509358,0.0652321428,0.0416002825,0.0443401765,0.0011267840,0.0070972490,0.0223458964,0.0209435690,0.0191830414,...,0.0284182140,0.0242827826,0.0193251842,0.0127239091,0.0150687459,0.0134124594,0.0207552391,0.0218085436,0.0172305872,0.0003558182
4,2022-08-14,0.0185000000,0.0594000000,0.0308412104,0.0462319885,0.0031213256,0.0102030010,0.0218000000,0.0216779206,0.0192293948,...,0.0324716866,0.0211000000,0.0226000000,0.0107576369,0.0093460002,0.0153375793,0.0201468040,0.0181031700,0.0186962536,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2023-09-17,0.0415209964,0.0826850964,0.0836655199,0.0953882875,0.0621944448,0.0250357518,0.0687114881,0.0843855193,0.0520837379,...,0.1256912758,0.0476282836,0.0378342803,0.0192114859,0.0272336308,0.0510918062,0.0488114704,0.0694055898,0.0511898924,0.0282080862
62,2023-09-24,0.0454263605,0.0652214592,0.0535924713,0.1104324461,0.0612637155,0.0241361816,0.0715342813,0.0913280741,0.0475794166,...,0.1086351073,0.0619353344,0.0290179062,0.0225148136,0.0477378878,0.0228047583,0.0411282197,0.0887502244,0.0333609978,0.0327421474
63,2023-10-01,0.0537000903,0.1068085532,0.0772468120,0.1139304323,0.0735736880,0.0255780550,0.0700379629,0.0800309871,0.0490688064,...,0.1791816723,0.0515910160,0.0452463454,0.0123147898,0.0359106170,0.0402288586,0.0553324688,0.0879154527,0.0499401448,0.0364127004
64,2023-10-08,0.0164014981,0.0191614214,0.0244221623,0.0285243798,0.0288575158,0.0067946342,0.0228243798,0.0199014981,0.0122671269,...,0.0313841068,0.0149907625,0.0089841068,0.0067152715,0.0086575158,0.0060529882,0.0121696285,0.0182841068,0.0101508940,0.0124301525


#### 1M : 1개월 단위

In [21]:
## 일별 적산
df_accu_resampled_1M = com_Prep.resample_by_last(df_accu, str_domain, 'METER_DATE', '1M')
df_accu_resampled_1M = df_accu_resampled_1M.reset_index().drop(columns = ['METER_DATE']).rename(columns = {str_domain + '_METER_DATE' : 'METER_DATE'})

str_file = 'KIER_' + str_domain + '_ACCU_1MONTH_InstBaseUpdated.csv' 
df_accu_resampled_1M.to_csv(str_dirName_h + str_file)
df_accu_resampled_1M

,METER_DATE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,HOT_HEAT_ACCU_561-2-2,HOT_HEAT_ACCU_561-2-3,HOT_HEAT_ACCU_561-2-4,HOT_HEAT_ACCU_561-3-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022-07-31,5.2298331444,11.8282410036,6.9409628122,7.7690230095,8.1070838688,2.4785151008,4.5648002824,3.3161990277,5.4690611614,...,11.1213442581,7.6877555695,4.7820541302,2.9587477383,3.2754375025,2.6883188530,4.2891311484,6.7208180004,4.8929090693,5.7512480679
1,2022-08-31,5.3109823589,12.1140213350,7.0977433808,7.9602606945,8.1263363787,2.5128410399,4.6751046536,3.4136220608,5.5623763217,...,11.2752462038,7.7880186370,4.8792092578,3.0055126806,3.3453840260,2.7590134819,4.3890476194,6.8225904668,4.9695805538,5.7534190956
2,2022-09-30,5.4116506107,12.4379050069,7.2989279513,8.2152066694,8.3008889950,2.5622122664,4.8390848598,3.5384361309,5.6798124668,...,11.4595123262,7.9182504216,4.9950984625,3.0552452724,3.4351785967,2.8511264011,4.5055826885,6.9828079797,5.0825850726,5.8301503361
3,2022-10-31,5.5710844986,12.8168631939,7.5868091358,8.5515701525,8.5698588859,2.6457116135,5.0844079747,3.7189522719,5.8483054232,...,11.7847964446,8.1310524631,5.1535769162,3.1456507658,3.5504270866,2.9781886449,4.6559844315,7.2484421148,5.2657871293,5.9465347146
4,2022-11-30,5.7791203872,13.2289866296,7.9111297173,8.9899042785,8.8671015767,2.7820883963,5.4402903381,3.9514666719,6.1478504596,...,12.1878516827,8.4379482455,5.4187530939,3.3002505828,3.7306306032,3.1578144949,4.8547484760,7.6138113912,5.5583036121,6.1644539202
5,2022-12-31,6.1201449096,13.6971395788,8.4660233118,9.6169459593,9.2893324074,2.9864661536,5.8991513317,4.3057289369,6.5785051025,...,12.8601718739,8.8512455504,5.7147377209,3.4799697915,4.1200548455,3.4154408188,5.1034120385,8.0945617735,5.9756088759,6.4395066655
6,2023-01-31,6.6885435924,14.3174431497,9.0084086350,10.3990608222,9.6971437920,3.1887199308,6.3826868496,4.7138148935,6.9465171919,...,13.7063830350,9.3528781655,6.0907451563,3.6881530033,4.4392241594,3.7015866913,5.3934119999,8.7145484446,6.4184241515,6.7081518783
7,2023-02-28,7.0665288617,14.9512836559,9.4686651780,10.9220001329,-3766.7847173094,3.3527641519,6.7756907857,5.0336961810,7.2785536253,...,14.4111297250,-3767.0614859137,6.3994005060,3.8409474150,-3772.1229690045,3.9583786824,5.6398546669,9.0976320662,6.8230844031,6.9785250491
8,2023-03-31,7.4181525226,15.6950741818,9.9254973236,11.4802429479,-3766.1906500633,3.5429371996,7.2146769062,5.3672256927,7.6652116576,...,15.2639169589,-3766.7314974455,6.7298515514,4.0054136857,-3771.8540839485,4.2351011175,5.9351052605,9.5144761535,7.2296853833,7.2448351941
9,2023-04-30,7.6133005684,16.1947909325,10.4434363859,12.0311371004,-3765.7437186234,3.6924689231,7.6112891612,5.6625107631,7.9872297767,...,16.0687567046,-3766.3624579615,7.0366296584,4.1563346237,-3771.6575867927,4.4602006703,6.2100115693,9.9251636290,7.5535381783,7.4812381129


In [22]:
## 월별 순시
df_inst_resampled_1M = pd.DataFrame()
df_inst_resampled_1M['METER_DATE'] = df_accu_resampled_1M['METER_DATE']

for str_h in df_h['HOUSE_LIST']:
    df_inst_resampled_1M[str_col_inst + '_' + str_h] = 0
    for i in range(0, len(df_accu_resampled_1M) - 1):
        df_inst_resampled_1M[str_col_inst + '_' + str_h].loc[i] = df_accu_resampled_1M[str_col_accu + '_' + str_h].loc[i + 1] - df_accu_resampled_1M[str_col_accu + '_' + str_h].loc[i]
str_file = 'KIER_' + str_domain + '_INST_1MONTH_InstBaseUpdated.csv' 
df_inst_resampled_1M.to_csv(str_dirName_h + str_file)
df_inst_resampled_1M

,METER_DATE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,HOT_HEAT_INST_561-1-4,HOT_HEAT_INST_561-2-1,HOT_HEAT_INST_561-2-2,HOT_HEAT_INST_561-2-3,HOT_HEAT_INST_561-2-4,HOT_HEAT_INST_561-3-1,...,HOT_HEAT_INST_563-22-3,HOT_HEAT_INST_563-22-4,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2
0,2022-07-31,0.0811492145,0.2857803314,0.1567805685,0.1912376849,0.0192525099,0.0343259391,0.1103043712,0.0974230332,0.0933151604,...,0.1539019457,0.1002630676,0.0971551276,0.0467649423,0.0699465235,0.0706946289,0.0999164711,0.1017724664,0.0766714845,0.0021710276
1,2022-08-31,0.1006682518,0.3238836719,0.2011845706,0.2549459749,0.1745526162,0.0493712266,0.1639802061,0.1248140701,0.1174361450,...,0.1842661224,0.1302317846,0.1158892047,0.0497325918,0.0897945707,0.0921129192,0.1165350691,0.1602175129,0.1130045188,0.0767312405
2,2022-09-30,0.1594338878,0.3789581870,0.2878811845,0.3363634831,0.2689698909,0.0834993470,0.2453231149,0.1805161410,0.1684929564,...,0.3252841184,0.2128020415,0.1584784537,0.0904054934,0.1152484900,0.1270622438,0.1504017430,0.2656341351,0.1832020567,0.1163843785
3,2022-10-31,0.2080358886,0.4121234357,0.3243205815,0.4383341260,0.2972426908,0.1363767828,0.3558823634,0.2325144000,0.2995450364,...,0.4030552381,0.3068957823,0.2651761777,0.1545998170,0.1802035166,0.1796258500,0.1987640445,0.3653692764,0.2925164828,0.2179192056
4,2022-11-30,0.3410245225,0.4681529492,0.5548935945,0.6270416808,0.4222308307,0.2043777573,0.4588609936,0.3542622650,0.4306546429,...,0.6723201911,0.4132973049,0.2959846270,0.1797192087,0.3894242423,0.2576263239,0.2486635625,0.4807503824,0.4173052638,0.2750527453
5,2022-12-31,0.5683986828,0.6203035709,0.5423853232,0.7821148629,0.4078113846,0.2022537772,0.4835355179,0.4080859566,0.3680120894,...,0.8462111611,0.5016326152,0.3760074354,0.2081832118,0.3191693139,0.2861458725,0.2899999614,0.6199866711,0.4428152756,0.2686452128
6,2023-01-31,0.3779852692,0.6338405062,0.4602565430,0.5229393106,-3776.4818611014,0.1640442211,0.3930039361,0.3198812875,0.3320364334,...,0.7047466901,-3776.4143640792,0.3086553496,0.1527944117,-3776.5621931639,0.2567919911,0.2464426670,0.3830836216,0.4046602516,0.2703731708
7,2023-02-28,0.3516236610,0.7437905259,0.4568321455,0.5582428151,0.5940672461,0.1901730477,0.4389861205,0.3335295116,0.3866580324,...,0.8527872339,0.3299884682,0.3304510454,0.1644662707,0.2688850561,0.2767224351,0.2952505936,0.4168440874,0.4066009801,0.2663101450
8,2023-03-31,0.1951480458,0.4997167507,0.5179390623,0.5508941525,0.4469314399,0.1495317235,0.3966122549,0.2952850705,0.3220181191,...,0.8048397457,0.3690394840,0.3067781070,0.1509209380,0.1964971558,0.2250995528,0.2749063088,0.4106874754,0.3238527950,0.2364029189
9,2023-04-30,0.2941240469,0.4064065545,0.4786420527,0.6221235210,0.4860694541,0.1247064106,0.3296738689,0.2305607781,0.2937867371,...,0.7253341566,0.3318491899,0.2559978265,0.1227194759,0.1922866220,0.1916315015,0.2547450568,0.3804981905,0.3097811029,0.2061103577


#### MAXACCU : 기간 총합

In [23]:
## 적분값 (총 사용량)
list_col = df_accu.columns[1:]
list_h = list_col.transpose()
df_accu_resampled_1M = pd.DataFrame()
df_accu_resampled_1M[str_col_accu] = df_accu[list_col].iloc[-1]
df_accu_resampled_1M['H_INDEX'] = list_h
df_accu_resampled_1M = df_accu_resampled_1M.reset_index()[[str_col_accu, 'H_INDEX']]

str_file = 'KIER_' + str_domain + '_ACCU_MAXACCU_InstBaseUpdated.csv' 
df_accu_resampled_1M = df_accu_resampled_1M.drop(index = 348, axis = 0)
df_accu_resampled_1M.to_csv(str_dirName_h + str_file)
df_accu_resampled_1M

,HOT_HEAT_ACCU,H_INDEX
0,8.7621799450,HOT_HEAT_ACCU_561-1-1
1,18.3302406914,HOT_HEAT_ACCU_561-1-2
2,12.4697291697,HOT_HEAT_ACCU_561-1-3
3,14.3572575678,HOT_HEAT_ACCU_561-1-4
4,-3763.9702938410,HOT_HEAT_ACCU_561-2-1
...,...,...
343,5.3126517676,HOT_HEAT_ACCU_563-23-2
344,7.3170235907,HOT_HEAT_ACCU_563-23-3
345,11.3675712354,HOT_HEAT_ACCU_563-23-4
346,8.7438420612,HOT_HEAT_ACCU_563-24-1
